## Doc2Vec

In [1]:
import gensim
from gensim.models import Doc2Vec

from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer=WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /Users/alex/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
tqdm.pandas()

In [4]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer=WordNetLemmatizer()
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


def preprocessing(j):
    result = re.sub(r'\d+', '', j)
    result = re.sub(r'[^\w\s]', '', result)
    result = j.strip()
    tokens = word_tokenize(j)
    result = [lemmatizer.lemmatize(i) for i in tokens if not i in stop_words and len(i) > 1 and not i.isdigit()]
    # is_noun = lambda pos: pos[:2] == 'NN'
    # nouns = [word for (word, pos) in nltk.pos_tag(result) if is_noun(pos)] 
    return result

In [9]:
def preprocess(st):
    st = st.lower()
    st = st.strip()
    text_tokens = word_tokenize(st)
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    filtered_sentence = (" ").join(tokens_without_sw)
    input_st = []
    for word in filtered_sentence:
        input_st.append(lemmatizer.lemmatize(word))
    sentence = ("").join(input_st)   
    blob = TextBlob(sentence)
    noun_phrases = blob.noun_phrases
    phrases = []
    for phrase in noun_phrases:
        phrases.append(phrase)
    return phrases
    # возвращает список всех nous phrases

In [5]:
df_d2v = pd.read_csv('/Users/alex/Desktop/ВШЭ/ML/Проекты/Project ML/df for proj.csv', sep=';')
df_d2v.head()

,Unnamed: 0,text,unique_tag
0,0,australia’s energy wars leave coal still stand...,environment
1,1,buying unique newborn baby gifts\n\nnot a long...,life
2,2,big bets on long shots\n\nexpenditures by the ...,sport and health
3,3,photo by charles deluvio on unsplash\n\ntools ...,sport and health
4,4,the greatest sin in marketing agencies\n\nlets...,work


In [6]:
train, test = train_test_split(df_d2v, test_size=0.3, random_state=42)

train_tagged = train.progress_apply(
    lambda r: TaggedDocument(words=preprocessing(r['text']), tags=[r.unique_tag]), axis=1)
test_tagged = test.progress_apply(
    lambda r: TaggedDocument(words=preprocessing(r['text']), tags=[r.unique_tag]), axis=1)

100%|██████████| 443/443 [00:02<00:00, 177.37it/s]


In [7]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [22]:
from tqdm import tqdm
model_dbow = Doc2Vec(dm=0, vector_size=400, negative=5, hs=0, min_count=1, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 1033/1033 [00:00<00:00, 1497137.54it/s]


In [23]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 1033/1033 [00:00<00:00, 3703176.10it/s]


CPU times: user 33.5 s, sys: 968 ms, total: 34.5 s
Wall time: 7.89 s


In [47]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [48]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

In [49]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

/Users/alex/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [50]:
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.16930022573363432
Testing F1 score: 0.16613384770083225


In [59]:
model_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=400, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)
model_dmm.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 1033/1033 [00:00<00:00, 1401719.84it/s]


In [60]:
%%time
for epoch in range(30):
    model_dmm.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dmm.alpha -= 0.002
    model_dmm.min_alpha = model_dmm.alpha

100%|██████████| 1033/1033 [00:00<00:00, 3953208.06it/s]


CPU times: user 1min 3s, sys: 597 ms, total: 1min 4s
Wall time: 14.5 s


In [61]:
y_train, X_train = vec_for_learning(model_dmm, train_tagged)
y_test, X_test = vec_for_learning(model_dmm, test_tagged)

In [62]:
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.47404063205417607
Testing F1 score: 0.46396115052063774


In [63]:
model_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
model_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
new_model = ConcatenatedDoc2Vec([model_dbow, model_dmm])

In [64]:
def get_vectors(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [65]:
y_train, X_train = get_vectors(new_model, train_tagged)
y_test, X_test = get_vectors(new_model, test_tagged)

In [66]:
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.4672686230248307
Testing F1 score: 0.4563127641705693
